In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating environment at `~/SSD/Code/Julia/tinympc-dev-julia/quadrotor_socp/Project.toml`


    Updating 

registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General`


   Installed ConcurrentUtilities ─ v2.3.1
   Installed Format ────────────── v1.3.6
   Installed Preferences ───────── v1.4.3
   Installed ChainRulesCore ────── v1.23.0


...

   Installed Libuuid_jll ───────── v2.39.3+0


   Installed OpenSSL ───────────── v1.4.2


   Installed TranscodingStreams ── v0.10.4


    Updating `~/SSD/Code/Julia/tinympc-dev-julia/quadrotor_socp/Project.toml`
  [0a1fb500] + BlockDiagonals v0.1.42
  [91a5bcdd] + Plots v1.39.0
    Updating `~/SSD/Code/Julia/tinympc-dev-julia/quadrotor_socp/Manifest.toml`
  [d1d4a3ce] + BitFlags v0.1.8
  [0a1fb500] + BlockDiagonals v0.1.42
  [d360d2e6] + ChainRulesCore v1.23.0
  [9e997f8a] + ChangesOfVariables v0.1.8
  [944b1d66] + CodecZlib v0.7.4
  [35d6a980] + ColorSchemes v3.24.0
  [3da002f7] + ColorTypes v0.11.4
  [c3611d14] + ColorVectorSpace v0.9.10
  [5ae59095] + Colors v0.12.10
  [34da2185] + Compat v4.14.0
  [f0e56b4a] + ConcurrentUtilities v2.3.1
  [187b0558] + ConstructionBase v1.5.4
  [d38c429a] + Contour v0.6.2
  [9a962f9c] + DataAPI v1.16.0
  [864edb3b] + DataStructures v0.18.17
  [ffbed154] + DocStringExtensions v0.9.3
  [460bff9d] + ExceptionUnwrapping v0.1.10
  [c87230d0] + FFMPEG v0.4.1
  [1a297f60] + FillArrays v1.9.3
  [26cc04aa] + FiniteDifferences v0.12.31
  [53c48c17] + FixedPointNumbers v0.8.4
  [1fa38f19] + 

  [05181044] + RelocatableFolders v1.0.1
  [ae029012] + Requires v1.3.0
  [708f8203] + Richardson v1.4.2
  [6c6a2e73] + Scratch v1.2.1
  [992d4aef] + Showoff v1.0.3
  [777ac1f9] + SimpleBufferStream v1.1.0
  [a2af1166] + SortingAlgorithms v1.2.1
  [276daf66] + SpecialFunctions v2.3.1
  [90137ffa] + StaticArrays v1.9.3
  [1e83bf80] + StaticArraysCore v1.4.2
  [82ae8749] + StatsAPI v1.7.0
  [2913bbd2] + StatsBase v0.34.2
  [62fd8b95] + TensorCore v0.1.1
  [3bb67fe8] + TranscodingStreams v0.10.4
  [5c2747f8] + URIs v1.5.1
  [1cfade01] + UnicodeFun v0.4.1
  [1986cc42] + Unitful v1.19.0
  [45397f5d] + UnitfulLatexify v1.6.3
  [41fe7b60] + Unzip v0.1.2
  [6e34b625] + Bzip2_jll v1.0.8+1
  [83423d85] + Cairo_jll v1.16.1+1
  [2702e6a9] + EpollShim_jll v0.0.20230411+0
  [2e619515] + Expat_jll v2.5.0+0
  [b22a6f82] + FFMPEG_jll v4.4.2+2
  [a3f928ae] + Fontconfig_jll v2.13.93+0
  [d7e528f0] + FreeType2_jll v2.13.1+0
  [559328eb] + FriBidi_jll v1.0.10+0
  [0656b61e] + GLFW_jll v3.3.9+0
  [d2c73de3]

In [2]:
using LinearAlgebra
using BlockDiagonals
using ForwardDiff
using Plots
using Random;
using Printf

In [ ]:
include("../LibTinyMPC_clean.jl")

In [ ]:
# Quadrotor
NSTATES = 6
NINPUTS = 3
NHORIZON = 30
NTOTAL = 201

# Initialize cache, settings, bounds, socs, workspace, solver

###### Cache
rho = 1e1
Kinf = zeros(NINPUTS,NSTATES)
Pinf = zeros(NSTATES,NSTATES)
Quu_inv = zeros(NINPUTS,NINPUTS)
AmBKt = zeros(NSTATES,NSTATES)
APf = zeros(NSTATES)
BPf = zeros(NINPUTS)
my_cache = TinyCache(rho, Kinf, Pinf, Quu_inv, AmBKt, APf, BPf)
#####

##### Settings
abs_pri_tol = 1e-2
abs_dua_tol = 1e-2
max_iter = 50
check_termination = 1
en_state_bound = 0 # ENABLE CONSTRAINTS OR NOT
en_input_bound = 1
en_state_soc = 0
en_input_soc = 0
my_stgs = TinySettings(abs_pri_tol, abs_dua_tol, max_iter, check_termination, en_state_bound, en_input_bound, en_state_soc, en_input_soc)
#####

###### Bounds
umin = -3*ones(NINPUTS, NHORIZON-1)
umax = 3*ones(NINPUTS, NHORIZON-1)
xmin = repeat([-1; -1; -1; -1; -1; -1], 1, NHORIZON)
xmax = repeat([1; 1; 1.0; 1; 1; 1], 1, NHORIZON)
z = zeros(NINPUTS, NHORIZON-1)
znew = zeros(NINPUTS, NHORIZON-1)
v = zeros(NSTATES, NHORIZON)
vnew = zeros(NSTATES, NHORIZON)
y = zeros(NINPUTS, NHORIZON-1)
g = zeros(NSTATES, NHORIZON)
my_bounds = TinyBounds(umin, umax, xmin, xmax, z, znew, v, vnew, y, g)
#####

##### SOCS (NOT USED IN THIS EXAMPLE)
ncu = 0
ncx = 0
cu = [0.4, 0]  # 0.3 is the thrust coefficient
cx = [0.6, 0]
qcu = [3, 0]  # 3 is the dimensions of 1st cone
qux = [3, 0]
Acu = [1, 0]
Acx = [1, 0]
zc = [zeros(NINPUTS, NHORIZON-1), zeros(NINPUTS, NHORIZON-1)]
vc = [zeros(NSTATES, NHORIZON), zeros(NSTATES, NHORIZON)]
zcnew = [zeros(NINPUTS, NHORIZON-1), zeros(NINPUTS, NHORIZON-1)]
vcnew = [zeros(NSTATES, NHORIZON), zeros(NSTATES, NHORIZON)]
yc = [zeros(NINPUTS, NHORIZON-1), zeros(NINPUTS, NHORIZON-1)]
gc = [zeros(NSTATES, NHORIZON), zeros(NSTATES, NHORIZON)]
my_socs = TinySocs(ncu, ncx, cu, cx, qcu, qux, Acu, Acx, zc, zcnew, vc, vcnew, yc, gc)
#####

##### Workspace
x = zeros(NSTATES, NHORIZON)
u = zeros(NINPUTS, NHORIZON-1)
q = zeros(NSTATES, NHORIZON)
r = zeros(NINPUTS, NHORIZON-1)
p = zeros(NSTATES, NHORIZON)
d = zeros(NINPUTS, NHORIZON-1)
pri_res_state = 1.0
pri_res_input = 1.0
dua_res_state = 1.0
dua_res_input = 1.0
status = 0
iter = 0
Q = 1e2*I(NSTATES) 
R = 1e1*I(NINPUTS)
#2D Double-integrator dynamics
h = 0.02 #50 Hz
temp_n = Int(NSTATES/2)
Adyn = [I(temp_n) h*I(temp_n); zeros(temp_n,temp_n) I(temp_n)]
Bdyn = [0.5*h*h*I(temp_n); h*I(temp_n)];
fdyn = zeros(NSTATES)

t = h*(0:NTOTAL-1)
Xref = zeros(NSTATES, NTOTAL)
# x0 = [4, 2, 20, -2, 3, -5.0]  # INITIAL STATE
x0 = [4, 2, 3, 1, -3, 3.0]
x[:,1] = 1*x0
xg = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

for k = 1:NTOTAL
  Xref[1:3,k] = sin(1*t[k])*2*ones(temp_n)
end
x0 .= Xref[:,1]

Uref = repeat([0; 0; 0.0], 1, NTOTAL-1) 

Xref_hrz = 1*Xref[:,1:NHORIZON]
Uref_hrz = 1*Uref[:,1:NHORIZON-1]
# Could provide a better reference from other solvers

Qu = zeros(NINPUTS, NINPUTS)
my_work = TinyWorkspace(x, u, q, r, p, d, pri_res_state, pri_res_input, dua_res_state, dua_res_input, status, iter, Q, R, Adyn, Bdyn, fdyn, Xref_hrz, Uref_hrz, Qu, my_bounds, my_socs)
#####

# Solver
solver = TinySolver(my_stgs, my_cache, my_work)